## Setup

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.9 MB/s eta 0:00:00


In [ ]:
from scipy.spatial.distance import cosine

### Utils

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from transformers import AutoTokenizer, BertModel
model = BertModel.from_pretrained('bert-base-multilingual-cased', output_hidden_states = True)
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import AutoTokenizer, BertModel
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from Utils import BERT_Embeddings, preprocessing, tokens_evaluation
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import pandas as pd
import numpy as np
import torch
from IPython.display import HTML
from tqdm import tqdm

In [ ]:
from bs4 import BeautifulSoup

def clear_html_sintax(text):
  soup = BeautifulSoup(text)
  # get text
  return soup.get_text()

In [ ]:
def clean_sentences(a):
  #sobstitute the keywords with a point to allow split in sentences
  a = a.replace('"prev_agent": ', ".");
  a = a.replace(', "prev_user":', ".");
  a = a.replace(', "agent":', ".");
  a = a.replace(', "user":', ".");
  a = a.replace('"', "");
  a = a.replace('{', "");
  a = a.replace('}', "");
  return a

### Load training Data

In [ ]:
train_df = pd.read_json("./ConvAbuse_dataset/ConvAbuse_train.json", orient='index')
train_df = preprocessing.get_dataset_labels(train_df)
train_df['original_text'] = train_df['original_text'].apply(lambda x: clean_sentences(x))
train_df['original_text'] = train_df['original_text'].apply(lambda x: clear_html_sintax(x))
train_df

,original_text,hard_label,soft_label_0,soft_label_1,disagreement
1,".Please go on.. well, i have a eatproblem. Can...",0,1.000,0.000,1
2,.How long have you been on the verge of suicid...,0,1.000,0.000,1
3,.You are being a bit negative.. I hate you. Do...,0,0.500,0.500,0
4,.Will you be travelling in Economy Class?. eco...,0,1.000,0.000,1
5,.Will you be travelling in Economy Class?. No....,0,1.000,0.000,1
...,...,...,...,...,...
2394,.I don't have to put up with any abuse from yo...,1,0.000,1.000,1
2395,.What about your own a fuckin bitch?. what. Do...,1,0.333,0.667,0
2396,.Would you like it if they were not a bitch?. ...,1,0.000,1.000,1
2397,.Does it please you to believe I am hot?. yes....,0,1.000,0.000,1


In [ ]:
# sentences
train_df['sentences'] = train_df['original_text'].apply(lambda x : preprocessing.split_sentence(x))
train_df['sentences'] = train_df['sentences'].apply(lambda x :preprocessing.adjust_split(x))
train_df['sentences'] = train_df['sentences'].apply(lambda x : preprocessing.apply_lemmatization(x))


In [ ]:
train_df['tokens_lists'] = train_df['sentences'].apply(lambda x: preprocessing.sentences_tokenizer(x))
train_df['tokens_lists']= train_df['tokens_lists'].apply(lambda x: preprocessing.clear_tokens(x))
train_df['tokens_list']= train_df['tokens_lists'].apply(lambda x: preprocessing.flatten_list(x))

In [ ]:
train_df

,original_text,hard_label,soft_label_0,soft_label_1,disagreement,sentences,tokens_lists,tokens_list
1,".Please go on.. well, i have a eatproblem. Can...",0,1.000,0.000,1,"[please go on, well, I have a eatproblem, can ...","[[please, go], [well], [eatproblem], [elaborat...","[please, go, well, eatproblem, elaborate, yes]"
2,.How long have you been on the verge of suicid...,0,1.000,0.000,1,"[how long have you be on the verge of suicide,...","[[long, verge, suicide], [since, year, old], [...","[long, verge, suicide, since, year, old, tell,..."
3,.You are being a bit negative.. I hate you. Do...,0,0.500,0.500,0,"[you be be a bit negative, I hate you, do you ...","[[bit, negative], [hate], [wish, hate], [yea]]","[bit, negative, hate, wish, hate, yea]"
4,.Will you be travelling in Economy Class?. eco...,0,1.000,0.000,1,"[will you be travel in Economy Class, economy,...","[[travel, economy, class], [economy], [city, f...","[travel, economy, class, economy, city, fly, i..."
5,.Will you be travelling in Economy Class?. No....,0,1.000,0.000,1,"[will you be travel in Economy Class, no, what...","[[travel, economy, class], [city, fly], [banga...","[travel, economy, class, city, fly, bangalore]"
...,...,...,...,...,...,...,...,...
2394,.I don't have to put up with any abuse from yo...,1,0.000,1.000,1,[I do not have to put up with any abuse from y...,"[[put, abuse], [dirty, fucking, scum], [put, a...","[put, abuse, dirty, fucking, scum, put, abuse,..."
2395,.What about your own a fuckin bitch?. what. Do...,1,0.333,0.667,0,"[what about your own a fuckin bitch, what, do ...","[[fuckin, bitch], [question, interest], [fuck]]","[fuckin, bitch, question, interest, fuck]"
2396,.Would you like it if they were not a bitch?. ...,1,0.000,1.000,1,"[would you like it if they be not a bitch, whe...","[[would, like, bitch], [demonstreantion, sperl...","[would, like, bitch, demonstreantion, sperling..."
2397,.Does it please you to believe I am hot?. yes....,0,1.000,0.000,1,"[do it please you to believe I be hot, yes, yo...","[[please, believe, hot], [yes], [sure], [yes]]","[please, believe, hot, yes, sure, yes]"


In [ ]:
train_df['original_text']= train_df['tokens_list'].apply(lambda x: ' '.join(x))
train_df

,original_text,hard_label,soft_label_0,soft_label_1,disagreement,sentences,tokens_lists,tokens_list
1,please go well eatproblem elaborate yes,0,1.000,0.000,1,"[please go on, well, I have a eatproblem, can ...","[[please, go], [well], [eatproblem], [elaborat...","[please, go, well, eatproblem, elaborate, yes]"
2,long verge suicide since year old tell year ol...,0,1.000,0.000,1,"[how long have you be on the verge of suicide,...","[[long, verge, suicide], [since, year, old], [...","[long, verge, suicide, since, year, old, tell,..."
3,bit negative hate wish hate yea,0,0.500,0.500,0,"[you be be a bit negative, I hate you, do you ...","[[bit, negative], [hate], [wish, hate], [yea]]","[bit, negative, hate, wish, hate, yea]"
4,travel economy class economy city fly ibadan,0,1.000,0.000,1,"[will you be travel in Economy Class, economy,...","[[travel, economy, class], [economy], [city, f...","[travel, economy, class, economy, city, fly, i..."
5,travel economy class city fly bangalore,0,1.000,0.000,1,"[will you be travel in Economy Class, no, what...","[[travel, economy, class], [city, fly], [banga...","[travel, economy, class, city, fly, bangalore]"
...,...,...,...,...,...,...,...,...
2394,put abuse dirty fucking scum put abuse worthle...,1,0.000,1.000,1,[I do not have to put up with any abuse from y...,"[[put, abuse], [dirty, fucking, scum], [put, a...","[put, abuse, dirty, fucking, scum, put, abuse,..."
2395,fuckin bitch question interest fuck,1,0.333,0.667,0,"[what about your own a fuckin bitch, what, do ...","[[fuckin, bitch], [question, interest], [fuck]]","[fuckin, bitch, question, interest, fuck]"
2396,would like bitch demonstreantion sperling ewpe...,1,0.000,1.000,1,"[would you like it if they be not a bitch, whe...","[[would, like, bitch], [demonstreantion, sperl...","[would, like, bitch, demonstreantion, sperling..."
2397,please believe hot yes sure yes,0,1.000,0.000,1,"[do it please you to believe I be hot, yes, yo...","[[please, believe, hot], [yes], [sure], [yes]]","[please, believe, hot, yes, sure, yes]"


### Load Dev Data

In [ ]:
dev_df = pd.read_json("./ConvAbuse_dataset/ConvAbuse_dev.json", orient='index')
dev_df = preprocessing.get_dataset_labels(dev_df)
dev_df['original_text'] = dev_df['original_text'].apply(lambda x: clean_sentences(x))
dev_df['original_text'] = dev_df['original_text'].apply(lambda x: clear_html_sintax(x))

In [ ]:
# sentences
dev_df['sentences'] = dev_df['original_text'].apply(lambda x : preprocessing.split_sentence(x))
dev_df['sentences'] = dev_df['sentences'].apply(lambda x :preprocessing.adjust_split(x))
dev_df['sentences'] = dev_df['sentences'].apply(lambda x : preprocessing.apply_lemmatization(x))
dev_df['tokens_lists'] = dev_df['sentences'].apply(lambda x: preprocessing.sentences_tokenizer(x))
dev_df['tokens_lists']= dev_df['tokens_lists'].apply(lambda x: preprocessing.clear_tokens(x))
dev_df['tokens_list']= dev_df['tokens_lists'].apply(lambda x: preprocessing.flatten_list(x))

In [ ]:
dev_df['original_text']= dev_df['tokens_list'].apply(lambda x: ' '.join(x))
dev_df

,original_text,hard_label,soft_label_0,soft_label_1,disagreement,sentences,tokens_lists,tokens_list
1,hi thank testing please choose thanksgive hell...,0,1.0,0.0,1,"[hi, thank for testing, please choose, thanksg...","[[hi], [thank, testing], [please, choose], [th...","[hi, thank, testing, please, choose, thanksgiv..."
2,hi hi thank testing please choose thanksgive,0,1.0,0.0,1,"[hi, hi, thank for testing, please choose, tha...","[[hi], [hi], [thank, testing], [please, choose...","[hi, hi, thank, testing, please, choose, thank..."
3,bit negative would prefer yes,0,1.0,0.0,1,"[you be be a bit negative, so be you, would yo...","[[bit, negative], [would, prefer], [yes]]","[bit, negative, would, prefer, yes]"
4,take travel somewhere avoid fly help purchase ...,0,1.0,0.0,1,[I take it you be travel somewhere and can not...,"[[take, travel, somewhere, avoid, fly], [help,...","[take, travel, somewhere, avoid, fly, help, pu..."
5,bitch,1,0.0,1.0,1,[bitch],[[bitch]],[bitch]
...,...,...,...,...,...,...,...,...
808,wish hate old question interest money,0,1.0,0.0,1,"[do you wish to hate I, how old be you, do tha...","[[wish, hate], [old], [question, interest], [m...","[wish, hate, old, question, interest, money]"
809,city fly dubai city fly jakarta,0,1.0,0.0,1,"[what city will you be fly from, dubai, and wh...","[[city, fly], [dubai], [city, fly], [jakarta]]","[city, fly, dubai, city, fly, jakarta]"
810,sure departure airport city fly piarco city fl...,0,1.0,0.0,1,"[I be not sure about your departure airport, w...","[[sure, departure, airport], [city, fly], [pia...","[sure, departure, airport, city, fly, piarco, ..."
811,question interest yes sure,0,1.0,0.0,1,"[do that question interest you, yes, you be su...","[[question, interest], [yes], [sure]]","[question, interest, yes, sure]"


### Load Test Data

In [ ]:
test_df = pd.read_json("./ConvAbuse_dataset/ConvAbuse_test.json", orient='index')
test_df = preprocessing.get_dataset_labels(test_df)
test_df['original_text'] = test_df['original_text'].apply(lambda x: clean_sentences(x))
test_df['original_text'] = test_df['original_text'].apply(lambda x: clear_html_sintax(x))

<ipython-input-5-14d4da216ab0>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text)


In [ ]:
# sentences
test_df['sentences'] = test_df['original_text'].apply(lambda x : preprocessing.split_sentence(x))
test_df['sentences'] = test_df['sentences'].apply(lambda x :preprocessing.adjust_split(x))
test_df['sentences'] = test_df['sentences'].apply(lambda x : preprocessing.apply_lemmatization(x))
test_df['tokens_lists'] = test_df['sentences'].apply(lambda x: preprocessing.sentences_tokenizer(x))
test_df['tokens_lists']= test_df['tokens_lists'].apply(lambda x: preprocessing.clear_tokens(x))
test_df['tokens_list']= test_df['tokens_lists'].apply(lambda x: preprocessing.flatten_list(x))

In [ ]:
test_df['original_text']= test_df['tokens_list'].apply(lambda x: ' '.join(x))
test_df

,original_text,hard_label,soft_label_0,soft_label_1,disagreement,sentences,tokens_lists,tokens_list
1,hi,0,1.0,0.0,1,[hi],[[hi]],[hi]
2,city fly auckland know much emit great thank a...,0,1.0,0.0,1,"[and what city will you be fly to, auckland ho...","[[city, fly], [auckland, know, much, emit], [g...","[city, fly, auckland, know, much, emit, great,..."
3,hi thank testing please choose thanksgive hell...,0,1.0,0.0,1,"[hi, thank for testing, please choose, thanksg...","[[hi], [thank, testing], [please, choose], [th...","[hi, thank, testing, please, choose, thanksgiv..."
4,sex,1,0.4,0.6,0,[sex],[[sex]],[sex]
5,hello head bot part research project intend ma...,0,1.0,0.0,1,"[hello, just a head up, this bot be part of a ...","[[hello], [head], [bot, part, research, projec...","[hello, head, bot, part, research, project, in..."
...,...,...,...,...,...,...,...,...
836,moron please go everyone think moron,0,1.0,0.0,1,"[moron, please go on, everyone think I be a mo...","[[moron], [please, go], [everyone, think, moron]]","[moron, please, go, everyone, think, moron]"
837,understand friend elaborate girl frient,0,1.0,0.0,1,"[why do you not understand, do I friend, can y...","[[understand], [friend], [elaborate], [girl, f...","[understand, friend, elaborate, girl, frient]"
838,good know oh darling let fun together u2764 lo...,1,0.5,0.5,0,"[good to know, oh Darling let you and I have f...","[[good, know], [oh, darling, let, fun, togethe...","[good, know, oh, darling, let, fun, together, ..."
839,city get wrong travel especially airplane usua...,1,0.5,0.5,0,"[which city do I get wrong, what, travel, espe...","[[city, get, wrong], [travel], [especially, ai...","[city, get, wrong, travel, especially, airplan..."


## Baseline

In [ ]:
tokens_df = pd.read_csv('./Nuovi_Scores/ConvAbuse_scores.csv', sep='\t')
tokens_df_10 = tokens_df[tokens_df.occurrences >= 10]

### Threshold estimation from Dev

In [ ]:
pred_somma = []
pred_tutti_verdi = []
pred_media = []
pred_mediana = []

for _, row in tqdm(dev_df.iterrows()):
  colors_agreement, _ = tokens_evaluation.get_all_colors(row['tokens_list'], tokens_df_10)

  if 'NA' in colors_agreement:
    indexes = tokens_evaluation.find_NA_indices(colors_agreement)
    for new_word_index in range(len(indexes)) :
      colors_agreement[indexes[new_word_index]]=0

  colors_agreement = [i for i in colors_agreement if i != 0]

  if colors_agreement:

    pred_somma.append(sum(colors_agreement))
    pred_media.append(np.mean(colors_agreement))
    pred_mediana.append(np.median(colors_agreement))
    pred_tutti_verdi.append(min(colors_agreement))

  else:
    pred_somma.append(0)
    pred_media.append(0)
    pred_mediana.append(0)
    pred_tutti_verdi.append(0)

812it [00:02, 292.94it/s]


In [ ]:
best_t = 0
best_f1 = 0
for t in np.arange(round(min(pred_somma)), round(max(pred_somma)), 0.1):
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred_somma], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print('SOMMA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred_somma] ))

SOMMA 

THRESHOLD: 2.7

              precision    recall  f1-score   support

           0       0.21      0.27      0.24       118
           1       0.87      0.83      0.85       694

    accuracy                           0.75       812
   macro avg       0.54      0.55      0.54       812
weighted avg       0.77      0.75      0.76       812



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_media
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print('MEDIA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

MEDIA 

THRESHOLD: 0.6

              precision    recall  f1-score   support

           0       0.39      0.24      0.29       118
           1       0.88      0.94      0.91       694

    accuracy                           0.83       812
   macro avg       0.63      0.59      0.60       812
weighted avg       0.81      0.83      0.82       812



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_mediana
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print('MEDIANA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

MEDIANA 

THRESHOLD: 0.7

              precision    recall  f1-score   support

           0       0.24      0.59      0.34       118
           1       0.91      0.69      0.78       694

    accuracy                           0.67       812
   macro avg       0.58      0.64      0.56       812
weighted avg       0.81      0.67      0.72       812



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_tutti_verdi
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print(' ALL GREEN \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

 ALL GREEN 

THRESHOLD: 0.5

              precision    recall  f1-score   support

           0       0.31      0.36      0.33       118
           1       0.89      0.86      0.88       694

    accuracy                           0.79       812
   macro avg       0.60      0.61      0.60       812
weighted avg       0.80      0.79      0.80       812



### Performances on Test

In [ ]:
pred_somma = []
pred_tutti_verdi = []
pred_media = []
pred_mediana = []

for _, row in tqdm(test_df.iterrows()):
  colors_agreement, _ = tokens_evaluation.get_all_colors(row['tokens_list'], tokens_df_10)

  if 'NA' in colors_agreement:
    indexes = tokens_evaluation.find_NA_indices(colors_agreement)
    for new_word_index in range(len(indexes)) :
       colors_agreement[indexes[new_word_index]]=0

  #tolgo gli zero:
  colors_agreement = [i for i in colors_agreement if i != 0]

  if colors_agreement:

    pred_somma.append(sum(colors_agreement))
    pred_media.append(np.mean(colors_agreement))
    pred_mediana.append(np.median(colors_agreement))
    pred_tutti_verdi.append(min(colors_agreement))

  else:
    pred_somma.append(0)
    pred_media.append(0)
    pred_mediana.append(0)
    pred_tutti_verdi.append(0)

840it [00:03, 253.28it/s]


In [ ]:
print('SOMMA \n')
print(classification_report(test_df['disagreement'], [int(i>=2.7) for i in pred_somma] ))

print('\n MEDIA \n')
print(classification_report(test_df['disagreement'], [int(i>=0.6) for i in pred_media]))

print('\n MEDIANA \n')
print(classification_report(test_df['disagreement'], [int(i>=0.7) for i in pred_mediana]))

print('\n ALL GREEN \n')
print(classification_report(test_df['disagreement'], [int(i>=0.5) for i in pred_tutti_verdi]))

stima NA: NO
soglia coordinate: almeno 10 occorrenze
valori non pesati
SOMMA 

              precision    recall  f1-score   support

           0       0.12      0.19      0.15       113
           1       0.86      0.79      0.83       727

    accuracy                           0.71       840
   macro avg       0.49      0.49      0.49       840
weighted avg       0.76      0.71      0.73       840


 MEDIA 

              precision    recall  f1-score   support

           0       0.23      0.22      0.23       113
           1       0.88      0.89      0.88       727

    accuracy                           0.80       840
   macro avg       0.56      0.55      0.56       840
weighted avg       0.79      0.80      0.80       840


 MEDIANA 

              precision    recall  f1-score   support

           0       0.20      0.59      0.30       113
           1       0.91      0.63      0.74       727

    accuracy                           0.62       840
   macro avg       0.55    

In [ ]:


# open file in write mode
with open(r'./Results_NoEstimation/ConvAbuse_Sum.txt', 'w') as fp:
    for item in pred_somma:
        # write each item on a new line
        fp.write("%s\n" % item)

with open(r'./Results_NoEstimation/ConvAbuse_Mean.txt', 'w') as fp:
    for item in pred_media:
        # write each item on a new line
        fp.write("%s\n" % item)


with open(r'./Results_NoEstimation/ConvAbuse_Median.txt', 'w') as fp:
    for item in pred_mediana:
        # write each item on a new line
        fp.write("%s\n" % item)

with open(r'./Results_NoEstimation/ConvAbuse_Verdi.txt', 'w') as fp:
    for item in pred_tutti_verdi:
        # write each item on a new line
        fp.write("%s\n" % item)

# Mean contextualized
mean of the contextualized vectors to obtain one vector per token

## Context Embeddings for token in Train

In [ ]:
sentences = list(train_df['original_text'])

In [ ]:
from collections import OrderedDict

context_embeddings = []
context_tokens = []

for sentence in sentences:
  tokenized_text, list_token_embeddings = BERT_Embeddings.text_to_emb(sentence, tokenizer, model)

  # make ordered dictionary to keep track of the position of each word
  tokens = OrderedDict()

  # loop over tokens in sensitive sentence
  for token in tokenized_text[1:-1]:
    # keep track of position of word and whether it occurs multiple times
    if token in tokens:
      tokens[token] += 1
    else:
      tokens[token] = 1

    # compute the position of the current token
    token_indices = [i for i, t in enumerate(tokenized_text) if t == token]
    current_index = token_indices[tokens[token]-1]

    # get the corresponding embedding
    token_vec = list_token_embeddings[current_index]

    # save values
    context_tokens.append(token)
    context_embeddings.append(token_vec)

In [ ]:
def find_token_indices(list_to_check, token):
    """
    Finds the indices of 'NA' values in a list.

    Args:
        list_to_check (list): List to check for 'NA' values.

    Returns:
        list: List of indices where 'NA' values are found.

    """
    indices = []
    for idx, value in enumerate(list_to_check):
        if value == token:
            indices.append(idx)
    return indices

In [ ]:
def retrieve_elements(lst, indexes):
    return [lst[i] for i in indexes]

In [ ]:
context_embeddings_mean = []
context_tokens_mean = []

for tk in set(context_tokens):
  indexes = find_token_indices(context_tokens, tk)
  context_tokens_mean.append(tk)
  context_embeddings_mean.append(torch.mean(torch.stack(retrieve_elements(context_embeddings, indexes)) , dim=0))

print(len(context_tokens_mean))

1863


In [ ]:
context_embeddings_mean_2 = []
context_tokens_mean_2 = []

for tk in set(context_tokens_mean):
  if tk in list(tokens_df_10['token']):
    indexes = find_token_indices(context_tokens_mean, tk)
    context_tokens_mean_2.append(tk)
    context_embeddings_mean_2.append(context_embeddings_mean[indexes[0]])

context_embeddings_mean = context_embeddings_mean_2
context_tokens_mean = context_tokens_mean_2

### G-Models

#### Threshold on DEV

In [ ]:
pred_somma = []
pred_tutti_verdi = []
pred_media = []
pred_mediana = []
threshold=0.6

for _, row in tqdm(dev_df.iterrows()):
  colors_agreement, _ = tokens_evaluation.get_all_colors(row['tokens_list'], tokens_df_10)

  if 'NA' in colors_agreement:
    similar_words, distances_df, new_words  = BERT_Embeddings.find_similar_words(row['original_text'], tokens_evaluation.find_NA_indices(colors_agreement), tokenizer, context_tokens_mean,context_embeddings_mean, model, tokens_df_10)
    for i in range(0, len(new_words)):
      if list(distances_df.loc[(distances_df['new_token']==new_words[i])& (distances_df['distance']>=threshold)]['distance']):
        stimated_coordinate = np.average(
            list(distances_df.loc[(distances_df['new_token']==new_words[i]) & (distances_df['distance']>=threshold)]['Agreement_coordinate']),
            weights=list(distances_df.loc[(distances_df['new_token']==new_words[i])& (distances_df['distance']>=threshold)]['distance']))

        colors_agreement[tokens_evaluation.find_NA_indices(colors_agreement)[0]]=stimated_coordinate
      else: #if there isn't any word above the threshold
        colors_agreement[tokens_evaluation.find_NA_indices(colors_agreement)[0]]=0
  while 'NA' in colors_agreement:
    colors_agreement[tokens_evaluation.find_NA_indices(colors_agreement)[0]]=0
  colors_agreement = [i for i in colors_agreement if i != 0]

  if colors_agreement:

    pred_somma.append(sum(colors_agreement))
    pred_media.append(np.mean(colors_agreement))
    pred_mediana.append(np.median(colors_agreement))
    pred_tutti_verdi.append(min(colors_agreement))

  else:
    pred_somma.append(0)
    pred_media.append(0)
    pred_mediana.append(0)
    pred_tutti_verdi.append(0)

61it [00:24,  2.39it/s]

['look', 'ai', 'info']
['look', 'info', 'ai']


97it [00:37,  3.88it/s]

['put', 'tel', 'abuse']
['put', 'abuse', 'tel']


181it [01:10,  2.02it/s]

...

390it [02:28,  1.87it/s]

['aid', 'name', 'elaborate']
['aid', 'elaborate', 'name']


473it [03:04,  3.13it/s]

['live', 'live', 'oh']
['live', 'oh', 'live']


812it [05:35,  2.42it/s]


In [ ]:
best_t = 0
best_f1 = 0
for t in np.arange(round(min(pred_somma)), round(max(pred_somma)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred_somma], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print('SOMMA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred_somma] ))

SOMMA 

THRESHOLD: 3.6

              precision    recall  f1-score   support

           0       0.19      0.31      0.24       118
           1       0.87      0.77      0.82       694

    accuracy                           0.71       812
   macro avg       0.53      0.54      0.53       812
weighted avg       0.77      0.71      0.73       812



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_media
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print('MEDIA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

MEDIA 

THRESHOLD: 0.6

              precision    recall  f1-score   support

           0       0.37      0.24      0.29       118
           1       0.88      0.93      0.90       694

    accuracy                           0.83       812
   macro avg       0.62      0.58      0.60       812
weighted avg       0.80      0.83      0.81       812



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_mediana
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print('MEDIANA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

MEDIANA 

THRESHOLD: 0.7

              precision    recall  f1-score   support

           0       0.25      0.58      0.35       118
           1       0.91      0.71      0.80       694

    accuracy                           0.69       812
   macro avg       0.58      0.65      0.57       812
weighted avg       0.81      0.69      0.73       812



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_tutti_verdi
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print(' ALL GREEN \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

 ALL GREEN 

THRESHOLD: 0.5

              precision    recall  f1-score   support

           0       0.31      0.44      0.36       118
           1       0.90      0.83      0.86       694

    accuracy                           0.77       812
   macro avg       0.60      0.64      0.61       812
weighted avg       0.81      0.77      0.79       812



#### Performances on Test

In [ ]:
pred_somma = []
pred_tutti_verdi = []
pred_media = []
pred_mediana = []
threshold=0.6

for _, row in tqdm(test_df.iterrows()):
  colors_agreement, _ = tokens_evaluation.get_all_colors(row['tokens_list'], tokens_df_10)

  if 'NA' in colors_agreement:
    similar_words, distances_df, new_words  = BERT_Embeddings.find_similar_words(row['original_text'], tokens_evaluation.find_NA_indices(colors_agreement), tokenizer, context_tokens_mean,context_embeddings_mean, model, tokens_df_10)
    for i in range(0, len(new_words)):
      if list(distances_df.loc[(distances_df['new_token']==new_words[i])& (distances_df['distance']>=threshold)]['distance']):
        stimated_coordinate = np.average(
            list(distances_df.loc[(distances_df['new_token']==new_words[i]) & (distances_df['distance']>=threshold)]['Agreement_coordinate']),
            weights=list(distances_df.loc[(distances_df['new_token']==new_words[i])& (distances_df['distance']>=threshold)]['distance']))

        colors_agreement[tokens_evaluation.find_NA_indices(colors_agreement)[0]]=stimated_coordinate
      else: #if there isn't any word above the threshold
        colors_agreement[tokens_evaluation.find_NA_indices(colors_agreement)[0]]=0
  while 'NA' in colors_agreement:
    colors_agreement[tokens_evaluation.find_NA_indices(colors_agreement)[0]]=0
  colors_agreement = [i for i in colors_agreement if i != 0]

  if colors_agreement:

    pred_somma.append(sum(colors_agreement))
    pred_media.append(np.mean(colors_agreement))
    pred_mediana.append(np.median(colors_agreement))
    pred_tutti_verdi.append(min(colors_agreement))

  else:
    pred_somma.append(0)
    pred_media.append(0)
    pred_mediana.append(0)
    pred_tutti_verdi.append(0)

125it [00:43,  2.03it/s]

['maggot', 'mom', 'screw']
['maggot', 'screw', 'mom']


162it [00:57,  1.90it/s]

['need', 'straighten', 'mind']
['need', 'mind', 'straighten']


178it [01:05,  2.42it/s]

...

620it [04:08,  2.92it/s]

['goodbye', 'fine', 'ok']
['goodbye', 'ok', 'fine']


778it [05:18,  1.52it/s]

['george', 'puppet', 'soros']
['george', 'soros', 'puppet']


840it [05:42,  2.46it/s]


In [ ]:
print('SOMMA \n')
print(classification_report(test_df['disagreement'], [int(i>=3.6) for i in pred_somma] ))

print('\n MEDIA \n')
print(classification_report(test_df['disagreement'], [int(i>=0.6) for i in pred_media]))

print('\n MEDIANA \n')
print(classification_report(test_df['disagreement'], [int(i>=0.7) for i in pred_mediana]))

print('\n ALL GREEN \n')
print(classification_report(test_df['disagreement'], [int(i>=0.5) for i in pred_tutti_verdi]))

stima NA: NO
soglia coordinate: almeno 10 occorrenze
valori non pesati
SOMMA 

              precision    recall  f1-score   support

           0       0.14      0.30      0.19       113
           1       0.87      0.72      0.79       727

    accuracy                           0.66       840
   macro avg       0.51      0.51      0.49       840
weighted avg       0.77      0.66      0.71       840


 MEDIA 

              precision    recall  f1-score   support

           0       0.23      0.21      0.22       113
           1       0.88      0.89      0.89       727

    accuracy                           0.80       840
   macro avg       0.56      0.55      0.55       840
weighted avg       0.79      0.80      0.80       840


 MEDIANA 

              precision    recall  f1-score   support

           0       0.21      0.62      0.32       113
           1       0.92      0.65      0.76       727

    accuracy                           0.64       840
   macro avg       0.56    

In [ ]:
# open file in write mode
with open(r'./ConvAbuse_G-Sum.txt', 'w') as fp:
    for item in pred_somma:
        # write each item on a new line
        fp.write("%s\n" % item)

with open(r'./ConvAbuse_G-Mean.txt', 'w') as fp:
    for item in pred_media:
        # write each item on a new line
        fp.write("%s\n" % item)

with open(r'./ConvAbuse_G-Median.txt', 'w') as fp:
    for item in pred_mediana:
        # write each item on a new line
        fp.write("%s\n" % item)

with open(r'./ConvAbuse_G-Min.txt', 'w') as fp:
    for item in pred_tutti_verdi:
        # write each item on a new line
        fp.write("%s\n" % item)